<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Deep-seek-R1-MedicalSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning DeepSeek-R1-Distill-Llama-8B

## Objective:
Adapt `DeepSeek-R1-Distill-Llama-8B` for medical chain-of-thought reasoning.

## Key Components:
- **Model:** `unsloth/DeepSeek-R1-Distill-Llama-8B`

> Add blockquote


- **Dataset:** 500 samples from `medical-o1-reasoning-SFT`
- **Tools:**
  - `Unsloth` (2x faster training)
  - 4-bit quantization
  - LoRA adapters
- **Result:** 44-minute training resulting in concise medical reasoning with structured `<think>` outputs.

## Performance Improvement:

| **Metric**         | **Before Fine-Tuning** | **After Fine-Tuning** |
|--------------------|------------------------|-----------------------|
| **Response Length** | 450 words              | 150 words             |
| **Reasoning Style** | Verbose                | Focused               |
| **Answer Format**   | Bulleted               | Paragraph             |


### step-by-step  fine-tune DeepSeek-R1-Distill-Llama-8B on medical data

##  1: Install All the Required Packages

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## 2. Model Initialization

In [ ]:
# Import the FastLanguageModel class from the unsloth library
# This library is likely optimized for efficient language model operations
from unsloth import FastLanguageModel

# Load a pre-trained language model and its corresponding tokenizer
# The model being loaded is "unsloth/DeepSeek-R1-Distill-Llama-8B"
# This is a distilled version of the Llama 8B model, optimized for faster inference
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Name of the pre-trained model
    max_seq_length=2048,  # Maximum sequence length the model can handle
    load_in_4bit=True,  # Load the model in 4-bit precision for memory efficiency
    token=hf_token  # Hugging Face token for authentication (e.g., for private models)
)

In [ ]:
# Apply Parameter-Efficient Fine-Tuning (PEFT) to the model using LoRA (Low-Rank Adaptation)
# This allows fine-tuning large models with fewer resources by only updating a small subset of parameters
model = FastLanguageModel.get_peft_model(
    model,  # The pre-trained model to which LoRA will be applied
    r=16,  # Rank of the low-rank matrices used in LoRA. Higher values increase capacity but also computational cost.
           # Suggested values: 8, 16, 32, 64, 128. Choose based on your task and resources.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # List of model layers to apply LoRA to.
                                                           # These are typically attention and feed-forward layers.
    lora_alpha=16,  # Scaling factor for LoRA weights. Controls the magnitude of updates.
                    # A higher value increases the impact of LoRA updates.
    lora_dropout=0,  # Dropout rate for LoRA layers. Set to 0 for optimal performance.
                     # Dropout can help prevent overfitting but is not necessary here.
    bias="none",  # Whether to include bias terms in LoRA. "none" is optimized for efficiency.
    use_gradient_checkpointing="unsloth",  # Enables gradient checkpointing to save memory during training.
                                           # "unsloth" is optimized for very long sequences and reduces VRAM usage by 30%.
    random_state=3407,  # Random seed for reproducibility. Ensures consistent results across runs.
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (RS-LoRA). Set to False by default.
    loftq_config=None,  # Configuration for LoftQ (if applicable). Set to None as it is not used here.
)

## 4. Dataset Preparation

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[0:500]",
    trust_remote_code=True
)

## 5. Prompt Formatting

In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for q, cot, ans in zip(examples["Question"], examples["Complex_CoT"], examples["Response"]):
        text = f"""Below is an instruction... [truncated prompt template]""" + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

## 6. LoRA Configuration



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16
)

## 7. Training Setup

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=60,
        fp16=True,
        output_dir="outputs"
    )
)

### 8. Start Training

In [ ]:
trainer.train()

## 9. Save & Deploy

In [ ]:
# Save locally
model.save_pretrained_merged("DeepSeek-R1-Medical-COT", tokenizer, save_method="merged_16bit")

# Push to Hub
model.push_to_hub_merged("username/DeepSeek-R1-Medical-COT", tokenizer, save_method="merged_16bit")